# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
import os
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heartDiseaseAutoML'

experiment=Experiment(ws, experiment_name)

In [3]:
#Compute CLuster
compute_cluster_name = 'project'
try:
    compute_cluster = ComputeTarget(workspace=ws,name=compute_cluster_name)
    print("Found existing compute cluster")
except ComputeTargetException:
    print("Creating new Compute Cluster")
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes = 4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_cluster_config)
    compute_cluster.wait_for_completion(show_output = True)
print(compute_cluster.get_status().serialize())

Creating new Compute Cluster
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-10T10:50:25.502000+00:00', 'errors': None, 'creationTime': '2021-01-10T10:50:19.835696+00:00', 'modifiedTime': '2021-01-10T10:50:35.780465+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
ds = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/GowthamiWudaru/heart-Disease-Prediction-With-Azure/main/heartDisease.csv")

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl config here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(
    task='classification',
    training_data=ds,
    label_column_name='num',
    n_cross_validations=5,
    model_explainability=True,
    compute_target = compute_cluster,
    **automl_settings
)

In [6]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
automl_best_run, fitted_model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()
for metric_name in automl_best_run_metrics:
    metric = automl_best_run_metrics[metric_name]
    print(metric_name, metric)

average_precision_score_macro 0.9071329207317993
accuracy 0.8487005649717514
f1_score_macro 0.8437302230446925
precision_score_macro 0.849678259472402
balanced_accuracy 0.8423458650558938
f1_score_micro 0.8487005649717514
f1_score_weighted 0.8475155037179182
AUC_macro 0.9002442263679896
log_loss 0.42849117830126204
recall_score_macro 0.8423458650558938
precision_score_weighted 0.8507473397563251
recall_score_weighted 0.8487005649717514
weighted_accuracy 0.8543117415780465
recall_score_micro 0.8487005649717514
AUC_micro 0.9045438092502154
AUC_weighted 0.9002442263679896
norm_macro_recall 0.6846917301117875
matthews_correlation 0.6919316663626404
precision_score_micro 0.8487005649717514
average_precision_score_micro 0.9087565196290919
average_precision_score_weighted 0.9093884098676368
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_7afbb833-3ef3-43b9-aae0-8d2ef144c9ad_27/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_7afbb833-3ef3-43b9-aae0-8d2ef1

In [9]:
automl_best_run.register_model('autoMLbestRun','outputs/model.pkl')

Model(workspace=Workspace.create(name='quick-starts-ws-134203', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-134203'), name=autoMLbestRun, id=autoMLbestRun:1, version=1, tags={}, properties={})

In [10]:
automl_best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/65_job_prep-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_c92231fbec862850c5704cc8e8183627f7cd1abd155a576554ddddf2911c7893_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/100_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/dataprep/python_span_5384a676-7be1-4542-a8ea-36aebd4a8a31.jsonl',
 'logs/azureml/dataprep/python_span_82333ddf-c3f9-446b-bf55-c5616dd1b40e.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [11]:
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')

In [12]:
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#hyperdrive has better accuracy. So doing the steps in hyperparameter_tuning.ipynb

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service